<a href="https://colab.research.google.com/github/MaryamAshraff2/PCOS-TRACKER/blob/main/GEMMA_3N_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [2]:
%%capture
# Install latest transformers for Gemma 3N
!pip install --no-deps git+https://github.com/huggingface/transformers.git # Only for Gemma 3N
!pip install --no-deps --upgrade timm # Only for Gemma 3N

Unsloth-
FastModel supports loading nearly any model now! This includes Vision, Text and Audio models!

In [3]:
from unsloth import FastModel
import torch

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3n-E4B-it-unsloth-bnb-4bit",
    "unsloth/gemma-3n-E2B-it-unsloth-bnb-4bit",
    # Pretrained models
    "unsloth/gemma-3n-E4B-unsloth-bnb-4bit",
    "unsloth/gemma-3n-E2B-unsloth-bnb-4bit",

    # Other Gemma 3 quants
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3n-E4B-it", # Or "unsloth/gemma-3n-E2B-it"
    dtype = None, # None for auto detection
    max_seq_length = 1024, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.7.7: Fast Gemma3N patching. Transformers: 4.54.0.dev0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Gemma3N does not support SDPA - switching to eager!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/3.72G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.15G [00:00<?, ?B/s]

...

preprocessor_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.70M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

Gemma 3N can handle multimodal inputs. We use Gemma 3N's recommended settings of temperature = 1.0, top_p = 0.95, top_k = 64

In [4]:
from transformers import TextStreamer
import gc
# Helper function for inference
def do_gemma_3n_inference(model, messages, max_new_tokens = 128):
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt = True, # Must add for generation
        tokenize = True,
        return_dict = True,
        return_tensors = "pt",
    ).to("cuda")
    _ = model.generate(
        **inputs,
        max_new_tokens = max_new_tokens,
        temperature = 1.0, top_p = 0.95, top_k = 64,
        streamer = TextStreamer(tokenizer, skip_prompt = True),
    )
    # Cleanup to reduce VRAM usage
    del inputs
    torch.cuda.empty_cache()
    gc.collect()

IMAGES

In [9]:
#SLOTH IMAGE USING WEB LINK
sloth_link = "https://files.worldwildlife.org/wwfcmsprod/images/Sloth_Sitting_iStock_3_12_2014/story_full_width/8l7pbjmj29_iStock_000011145477Large_mini__1_.jpg"

messages = [{
    "role" : "user",
    "content": [
        { "type": "image", "image" : sloth_link },
        { "type": "text",  "text" : "Which films does this animal feature in?" }
    ]
}]
# You might have to wait 1 minute for Unsloth's auto compiler
do_gemma_3n_inference(model, messages, max_new_tokens = 256)

This adorable animal is a **sloth**, and it has featured in several films! Here are some notable ones:

* **Zootopia (2016):** Flash, a sloth, is one of the most beloved characters in this Disney movie. He works at the DMV and provides a humorous and heartwarming element to the film.
* **Ice Age (2002):** While not a major character, sloths are briefly seen in this animated film.
* **The Jungle Book 2 (1998):** A sloth appears in this Disney animated film. 

Sloths have become popular characters in animation and live-action films due to their unique and slow-moving nature, often used for comedic effect or to represent patience and relaxation. 

<end_of_turn>


In [10]:
#MINGYU IMAGE UPLOADED FROM LOCAL DEVICE
from google.colab import files
uploaded_files = files.upload()

image_path = list(uploaded_files.keys())[0]
print("Uploaded image path:", image_path)

messages = [{
    "role": "user",
    "content": [
        { "type": "image", "image": image_path },
        { "type": "text", "text": "Describe this image." }
    ]
}]

do_gemma_3n_inference(model, messages, max_new_tokens=256)

Saving 55b01a20bb34086fa47e75494beab72b.jpg to 55b01a20bb34086fa47e75494beab72b (1).jpg
Uploaded image path: 55b01a20bb34086fa47e75494beab72b (1).jpg
A medium shot captures a handsome Asian man, likely in his late twenties or early thirties, smiling warmly and looking slightly off-camera to his right. He's dressed in a smart casual style, sporting a dark navy blue blazer over a light blue and white vertical striped shirt. The shirt's collar is open, revealing a glimpse of a patterned undershirt with small, colorful floral designs. 

His dark, neatly styled hair is slightly tousled, and his facial expression is pleasant and approachable. He has a subtle smile, showing just a hint of his upper teeth. 

The background is softly blurred, suggesting a crowd of people, possibly at an event or public gathering. The overall lighting appears natural and soft, highlighting his features without harsh shadows. The focus is sharply on the man, making him the clear subject of the image.<end_of_turn>

In [12]:
#MY IMAGE UPLOADED FROM LOCAL DEVICE
from google.colab import files
uploaded_files = files.upload()

image_path = list(uploaded_files.keys())[0]
# print("Uploaded image path:", image_path)

messages = [{
    "role": "user",
    "content": [
        { "type": "image", "image": image_path },
        { "type": "text", "text": "Write a short poem inspired by this person's smile." }
    ]
}]

do_gemma_3n_inference(model, messages, max_new_tokens=256)

Saving WhatsApp Image 2025-03-30 at 14.06.46_1953b14c.jpg to WhatsApp Image 2025-03-30 at 14.06.46_1953b14c (1).jpg
A gentle curve, a warm delight,
A smile that catches fading light.
Beneath the trees, a peaceful scene,
A happy heart, serene and keen. 
A quiet joy, a lovely grace,
Reflected in a smiling face. 



<end_of_turn>


AUDIO

In [14]:
from google.colab import files

# Prompt you to upload your audio file
uploaded = files.upload()

# Get the uploaded filename
audio_file = list(uploaded.keys())[0]
print(f"✅ Uploaded file: {audio_file}")


Saving WhatsApp Audio 2025-07-22 at 21.06.34_9715b798.mp3 to WhatsApp Audio 2025-07-22 at 21.06.34_9715b798.mp3
✅ Uploaded file: WhatsApp Audio 2025-07-22 at 21.06.34_9715b798.mp3


In [17]:
# Choose your output language
language = "Tamil"  # Change this to "Tamil", "Sinhala", "English", etc.

# Create the message to send to Gemma 3N
messages = [{
    "role": "user",
    "content": [
        { "type": "audio", "audio": audio_file },
        { "type": "text",  "text": f"Please transcribe this audio in english and translate it into {language}." }
    ]
}]

# Inference call (assuming your model is already loaded)
do_gemma_3n_inference(model, messages, max_new_tokens=256)

/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Here's the transcription in English and the translation in Tamil:

**English Transcription:**

Hi, my name is Maryam. I'm recording this message to test if AI can understand my voice. Let's see if it can transcribe what I say and translate it to another language. Maybe Urdu or Tamil. Right now we are working on a PCOS detection and health guide application. That's it. Thank you. Bye.

**Tamil Translation:**

வணக்கம், என் பெயர் மரியா. நான் இந்த செய்தியை செயற்கை நுண்ணறிவு எனது குரலைப் புரிந்து கொள்ளுமா என்று சோதிக்க பதிவு செய்கிறேன். நான் சொல்வதைப் புரிந்துகொண்டு மற்றொரு மொழிக்கு மொழிபெயர்க்க முடியுமா என்று பார்ப்போம். ஒருவேளை உருது அல்லது தமிழ். இப்போது நாங்கள் PCOS கண்டறிதல் மற்றும் சுகாதார வழிகாட்டி பயன்பாட்டில் பணிபுரிந்து வருகிறோம். அவ்வளவுதான். நன்றி. போய் வருகிறேன்.<end_of_turn>


In [18]:
from google.colab import files
uploaded = files.upload()
audio_file = list(uploaded.keys())[0]
print(f"Uploaded file: {audio_file}")


# Create the message to send to Gemma 3N
messages = [{
    "role": "user",
    "content": [
        { "type": "audio", "audio": audio_file },
        { "type": "text",  "text": f"Listen to this song and describe the overall mood or vibe. "
                "What emotions does it convey? And what would be the perfect situation, place, or moment to listen to this track?" }
    ]
}]

# Inference call (assuming your model is already loaded)
do_gemma_3n_inference(model, messages, max_new_tokens=256)

Saving keshi - Soft Spot (Official Music Video).mp3 to keshi - Soft Spot (Official Music Video).mp3
Uploaded file: keshi - Soft Spot (Official Music Video).mp3
The overall mood of this song is **melancholic and introspective**. It carries a sense of **loneliness and yearning**, tinged with a quiet **resignation**. There's a feeling of looking back on something with a mix of sadness and acceptance.

The emotions it conveys are primarily **sadness, longing, and a touch of wistfulness**. It feels like a reflection on a past relationship or a lost connection, carrying a weight of unspoken words and missed opportunities. There's also a subtle sense of **peace** in the acceptance of this feeling.

The perfect situation, place, or moment to listen to this track would be during a **quiet evening alone**, perhaps while **contemplating past memories**. It would be ideal for moments of **reflection, introspection, or simply feeling a little emotionally drained**. It's a song for those who appreci

In [19]:
messages = [{
    "role" : "user",
    "content": [
        { "type": "audio", "audio" : audio_file },
        { "type": "image", "image" : image_path },
        { "type": "text",  "text" : "What is this audio and image about? "\
                                    "How are they related?" }
    ]
}]
do_gemma_3n_inference(model, messages, max_new_tokens = 256)

The audio is a song titled "Ado Ko Aw" by the Turkish singer Sefo. The lyrics express feelings of loneliness and longing for a loved one. 

The image shows a young woman wearing a hijab, a coat, and a scarf, posing outdoors amidst pine trees. She appears to be enjoying a peaceful moment in nature. 

The audio and image are related thematically through the feeling of solitude and contemplation. While the song expresses a more intense emotional state of longing, the image captures a moment of quiet reflection in a natural setting, which can also evoke feelings of introspection and perhaps a sense of being alone with one's thoughts. 

It's possible the user who shared these elements found a connection between the melancholic mood of the song and the serene atmosphere of the image, or perhaps they simply appreciate both as individual pieces of art.<end_of_turn>


FINETUNE

In [20]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # Should leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.model.language_model` require gradients


In [22]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

In [24]:
from datasets import load_dataset
dataset = load_dataset("mlabonne/FineTome-100k", split = "train[:3000]")

README.md:   0%|          | 0.00/982 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/117M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [25]:
from unsloth.chat_templates import standardize_data_formats
dataset = standardize_data_formats(dataset)

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/3000 [00:00<?, ? examples/s]

In [26]:
dataset[100]

{'conversations': [{'content': 'What is the modulus operator in programming and how can I use it to calculate the modulus of two given numbers?',
   'role': 'user'},
  {'content': 'In programming, the modulus operator is represented by the \'%\' symbol. It calculates the remainder when one number is divided by another. To calculate the modulus of two given numbers, you can use the modulus operator in the following way:\n\n```python\n# Calculate the modulus\nModulus = a % b\n\nprint("Modulus of the given numbers is: ", Modulus)\n```\n\nIn this code snippet, the variables \'a\' and \'b\' represent the two given numbers for which you want to calculate the modulus. By using the modulus operator \'%\', we calculate the remainder when \'a\' is divided by \'b\'. The result is then stored in the variable \'Modulus\'. Finally, the modulus value is printed using the \'print\' statement.\n\nFor example, if \'a\' is 10 and \'b\' is 4, the modulus calculation would be 10 % 4, which equals 2. Theref

In [27]:
def formatting_prompts_func(examples):
   convos = examples["conversations"]
   texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False).removeprefix('<bos>') for convo in convos]
   return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True)

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [28]:
dataset[100]["text"]

'<start_of_turn>user\nWhat is the modulus operator in programming and how can I use it to calculate the modulus of two given numbers?<end_of_turn>\n<start_of_turn>model\nIn programming, the modulus operator is represented by the \'%\' symbol. It calculates the remainder when one number is divided by another. To calculate the modulus of two given numbers, you can use the modulus operator in the following way:\n\n```python\n# Calculate the modulus\nModulus = a % b\n\nprint("Modulus of the given numbers is: ", Modulus)\n```\n\nIn this code snippet, the variables \'a\' and \'b\' represent the two given numbers for which you want to calculate the modulus. By using the modulus operator \'%\', we calculate the remainder when \'a\' is divided by \'b\'. The result is then stored in the variable \'Modulus\'. Finally, the modulus value is printed using the \'print\' statement.\n\nFor example, if \'a\' is 10 and \'b\' is 4, the modulus calculation would be 10 % 4, which equals 2. Therefore, the ou

TRAIN MODEL

In [29]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "paged_adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/3000 [00:00<?, ? examples/s]

In [30]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

Map (num_proc=2):   0%|          | 0/3000 [00:00<?, ? examples/s]

In [31]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

'<bos><start_of_turn>user\nWhat is the modulus operator in programming and how can I use it to calculate the modulus of two given numbers?<end_of_turn>\n<start_of_turn>model\nIn programming, the modulus operator is represented by the \'%\' symbol. It calculates the remainder when one number is divided by another. To calculate the modulus of two given numbers, you can use the modulus operator in the following way:\n\n```python\n# Calculate the modulus\nModulus = a % b\n\nprint("Modulus of the given numbers is: ", Modulus)\n```\n\nIn this code snippet, the variables \'a\' and \'b\' represent the two given numbers for which you want to calculate the modulus. By using the modulus operator \'%\', we calculate the remainder when \'a\' is divided by \'b\'. The result is then stored in the variable \'Modulus\'. Finally, the modulus value is printed using the \'print\' statement.\n\nFor example, if \'a\' is 10 and \'b\' is 4, the modulus calculation would be 10 % 4, which equals 2. Therefore, t

In [32]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

'                               In programming, the modulus operator is represented by the \'%\' symbol. It calculates the remainder when one number is divided by another. To calculate the modulus of two given numbers, you can use the modulus operator in the following way:\n\n```python\n# Calculate the modulus\nModulus = a % b\n\nprint("Modulus of the given numbers is: ", Modulus)\n```\n\nIn this code snippet, the variables \'a\' and \'b\' represent the two given numbers for which you want to calculate the modulus. By using the modulus operator \'%\', we calculate the remainder when \'a\' is divided by \'b\'. The result is then stored in the variable \'Modulus\'. Finally, the modulus value is printed using the \'print\' statement.\n\nFor example, if \'a\' is 10 and \'b\' is 4, the modulus calculation would be 10 % 4, which equals 2. Therefore, the output of the above code would be:\n\n```\nModulus of the given numbers is: 2\n```\n\nThis means that the modulus of 10 and 4 is 2.<end_of_t

In [33]:
# @title Show current memory stats
import gc
gc.collect()
torch.cuda.empty_cache()
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
12.592 GB of memory reserved.


In [34]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 19,210,240 of 7,869,188,432 (0.24% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,7.837900
2,7.321300
3,5.610800
4,6.834400
5,6.056300
6,5.708100
7,5.881300
8,6.727000
9,6.340400
10,6.253800


In [35]:
# @title Show final memory and time stats
gc.collect()
torch.cuda.empty_cache()
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

632.7072 seconds used for training.
10.55 minutes used for training.
Peak reserved memory = 12.592 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 85.422 %.
Peak reserved memory for training % of max memory = 0.0 %.


In [36]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)
messages = [{
    "role": "user",
    "content": [{
        "type" : "text",
        "text" : "Continue the sequence: 1, 1, 2, 3, 5, 8,",
    }]
}]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
    tokenize = True,
    return_dict = True,
).to("cuda")
outputs = model.generate(
    **inputs,
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
)
tokenizer.batch_decode(outputs)

['<bos><start_of_turn>user\nContinue the sequence: 1, 1, 2, 3, 5, 8,<end_of_turn>\n<start_of_turn>model\n13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6']

In [37]:
messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "Why is the sky blue?",}]
}]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
    tokenize = True,
    return_dict = True,
).to("cuda")

from transformers import TextStreamer
_ = model.generate(
    **inputs,
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

The sky is blue because of a phenomenon called Rayleigh scattering. Sunlight is made up of all the colors of the rainbow, but blue light has a shorter wavelength than red light. When sunlight enters the Earth's atmosphere, it collides with tiny air molecules (mostly nitrogen and oxygen). This collision causes the blue light to


In [38]:
model.save_pretrained("gemma-3n")  # Local saving
tokenizer.save_pretrained("gemma-3n")
# model.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving
# tokenizer.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving

['gemma-3n/processor_config.json']

In [ ]:
if True:
    from unsloth import FastModel
    model, tokenizer = FastModel.from_pretrained(
        model_name = "gemma-3n", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        load_in_4bit = True,
        device_map="cuda", # Explicitly set device_map to cuda
    )

messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "What is Gemma-3N?",}]
}]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
    tokenize = True,
    return_dict = True,
).to("cuda")

from transformers import TextStreamer
_ = model.generate(
    **inputs,
    max_new_tokens = 128, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

==((====))==  Unsloth 2025.7.7: Fast Gemma3N patching. Transformers: 4.54.0.dev0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Gemma3N does not support SDPA - switching to eager!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [1]:
if True: # Change to True to save finetune!
    model.save_pretrained_merged("gemma-3N-finetune", tokenizer)

NameError: name 'model' is not defined

In [2]:
if True: # Change to True to upload finetune
    model.push_to_hub_merged(
        "HF_ACCOUNT/gemma-3N-finetune", tokenizer,
        token = "hf_..."
    )

NameError: name 'model' is not defined

In [ ]:
if True: # Change to True to save to GGUF
    model.save_pretrained_gguf(
        "gemma-3N-finetune",
        quantization_type = "Q8_0", # For now only Q8_0, BF16, F16 supported
    )

In [ ]:
if True: # Change to True to upload GGUF
    model.push_to_hub_gguf(
        "gemma-3N-finetune",
        quantization_type = "Q8_0", # Only Q8_0, BF16, F16 supported
        repo_id = "HF_ACCOUNT/gemma-3N-finetune-gguf",
        token = "hf_...",
    )